The notebook is meant to reqrite hdf5 files, specifically the COMPAS_Output_wWeights_SM_2.h5 file in order to add WD+WD to the BSE_Double_Compact objects to eventually calculate the rate of WD+WD,

My steps to do this include:,
- Making the DCO table shorter so that it only encompasses things that I actually need,
- Making an empty table that has the same columns as the DCO table,
- Masking the original system parameters so that I only select for WD+WD that merge within a Hubble time,
    - Computing the coalescence time,
    - Checking if the binary merges within a hubble time,
- Adding all the information about these WD+WD to the empty table and then adding this table to the original DCO table,
- Using this new table to create a new COMPAS output to then run fastcosmicintegration on,
- Changing FastCosmisIntegration.py so that BWD are added as a DCO type + ,
- run FastCosmicIntegration via the blackbox method and compare with the Brill Paper Lieke sent"

Let's first look at the columns in BSE_DCOs to see what information we need of the WD+WD to gather

In [2]:
#importing, make sure the kernel is correct or the module will be unknown
import h5py as h5
import pandas as pd
from astropy.table import Table
from astropy import units as u
from astropy import constants as const
import numpy as np
import matplotlib.pyplot as plt
plt.rc('text.latex', preamble=r'\usepackage{textgreek}')

In [3]:
import sys
import os

# Add the subdir to sys.path
sys.path.append('/home/jovyan/home/research_work/useful_py_scripts/')

# Now you can import the module
import useful_fncs 


In [4]:
# reading in the HDF5 file - this file is the AIS file that we should expect many WDs in
pathToWDWD_edit_H5 = '/home/jovyan/home/copy_h5_files/v03.08.01/COMPAS_Output_wWeights_fbin0.7.h5' #path of the hdf5 file
Data_WD_edit = h5.File(pathToWDWD_edit_H5,"r")

In [5]:
# we want to read in the bse_system_paramtetrs to get information about thes different systems
DCO_OG = pd.DataFrame() # making a pandas dataframe
for key in Data_WD_edit["BSE_Double_Compact_Objects"].keys(): #looping through the \"keys\" or paramters in BSE_System_Parameters
    DCO_OG[key] = Data_WD_edit["BSE_Double_Compact_Objects"][key][()] # adding these columns to the dataframe"

# we want to read in the bse_system_paramtetrs to get information about thes different systems
WD_SP_edit = pd.DataFrame() # making a pandas dataframe
for key in Data_WD_edit["BSE_System_Parameters"].keys(): #looping through the "keys" or paramters in BSE_System_Parameters
    WD_SP_edit[key] = Data_WD_edit["BSE_System_Parameters"][key][()] # adding these columns to the dataframe

# closing the hdf5 file to prevent corruption later
Data_WD_edit.close()

/tmp/ipykernel_160335/2516234215.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  WD_SP_edit[key] = Data_WD_edit["BSE_System_Parameters"][key][()] # adding these columns to the dataframe
/tmp/ipykernel_160335/2516234215.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  WD_SP_edit[key] = Data_WD_edit["BSE_System_Parameters"][key][()] # adding these columns to the dataframe
/tmp/ipykernel_160335/2516234215.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many time

In [14]:
pd.set_option('display.max_columns', None) # allows us to see all of the columns
display(DCO_OG)

,Coalescence_Time,Eccentricity@DCO,Mass(1),Mass(2),Merges_Hubble_Time,Record_Type,Recycled_NS(1),Recycled_NS(2),SEED,SemiMajorAxis@DCO,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight
0,1.037345e+07,5.332106e-03,37.178948,28.196823,0,1,0,0,50,1.219466,14,14,4.339925,0.501814
1,3.346435e+26,6.691322e-01,9.122940,39.417105,0,1,0,0,69,109625.172735,14,14,3.990566,2.000000
2,8.312348e+15,1.570092e-16,33.856725,20.400369,0,1,0,0,117,176.427128,14,14,4.892572,2.000000
3,9.817088e+18,4.513478e-01,15.792861,7.879055,0,1,0,0,151,668.408497,14,14,9.962568,2.000000
4,9.473409e+15,7.356808e-01,11.070854,11.464652,0,1,0,0,154,188.842828,14,14,7.186891,0.791100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26014,5.561402e+01,2.237726e-16,14.462566,9.161675,1,1,0,0,99990,0.027126,14,14,7.248772,0.016086
26015,1.571516e+05,2.237726e-16,19.761078,11.276390,0,1,0,0,99991,0.241127,14,14,6.682860,0.119750
26016,4.195129e+09,4.460365e-16,19.694595,14.726406,0,1,0,0,99992,3.378335,14,14,5.907518,0.146851
26017,1.355348e+02,1.584931e-01,6.341612,3.450145,1,1,0,0,99994,0.017726,14,14,10.245794,0.053820


Let's drop any columns we don't think are necessary to have anymore

In [7]:
# DCO_OG = DCO_OG.drop(['dmMT(1)','dmMT(2)','dmWinds(1)','dmWinds(2)'],axis=1)

Let's list the columns of this DCO table

In [8]:
dco_columns = list(DCO_OG.columns)

Now that we see what columns are needed for BSE_DCOs let's look at BSE_System_Parameters to look at what information we already have

In [15]:
pd.set_option('display.max_columns', None) # allows us to see all of the columns
display(WD_SP_edit)

,CH_on_MS(1),CH_on_MS(2),Eccentricity@ZAMS,Equilibrated_At_Birth,Error,Evolution_Status,Mass@ZAMS(1),Mass@ZAMS(2),Merger,Merger_At_Birth,Metallicity@ZAMS(1),Metallicity@ZAMS(2),Omega@ZAMS(1),Omega@ZAMS(2),PO_Add_Options_To_SysParms,PO_Allow_Immediate_RLOF>CE_To_Survive_CE,PO_Allow_MS_To_Survive_CE,PO_Allow_Non_Stripped_ECSN,PO_Allow_RLOF@Birth,PO_Allow_Radiative_Envelope_To_Survive_CE,PO_Allow_Touching@Birth,PO_BB_Mass_xFer_Stblty_Prscrptn,PO_BH_Kicks_Mode,PO_CE_Alpha,PO_CE_Alpha_Thermal,PO_CE_Formalism,PO_CE_Lambda,PO_CE_Lambda_Multiplier,PO_CE_Lambda_Prscrptn,PO_CE_Mass_Accr_Constant,PO_CE_Mass_Accr_Max,PO_CE_Mass_Accr_Min,PO_CE_Mass_Accr_Prscrptn,PO_CE_Recomb_Enrgy_Dnsty,PO_CE_Slope_Kruckow,PO_CHE_Mode,PO_Check_Photon_Tiring_Limit,PO_Circularise@MT,PO_Conserve_AngMom@Circ,PO_Cool_WindMassLoss_Multipl,PO_Eccentricity,PO_Eccentricity_Dstrbtn,PO_Eccentricity_Dstrbtn_Max,PO_Eccentricity_Dstrbtn_Min,PO_Eddington_Accr_Factor,PO_Enable_Rotationally_Enhanced_Mass_Loss,PO_Enhance_CHE_lifetimes_luminosities,PO_Envelope_State_Prscrptn,PO_Evolution_Mode,PO_FP_Error_Mode,PO_Fryer22_crit_COcore_Mass,PO_Fryer22_mixing_fraction,PO_Fryer_SN_Engine,PO_Initial_Mass,PO_Initial_Mass(1),PO_Initial_Mass(2),PO_Initial_Mass_Func_Max,PO_Initial_Mass_Func_Min,PO_Initial_Mass_Func_Power,PO_Initial_Mass_Function,PO_Kick_Direction_Dstrbtn,PO_Kick_Direction_Power,PO_Kick_Magnitude,PO_Kick_Magnitude(1),PO_Kick_Magnitude(2),PO_Kick_Magnitude_Dstrbtn,PO_Kick_Magnitude_Dstrbtn_Max,PO_Kick_Magnitude_Random,PO_Kick_Magnitude_Random(1),PO_Kick_Magnitude_Random(2),PO_Kick_Mean_Anomaly(1),PO_Kick_Mean_Anomaly(2),PO_Kick_Phi(1),PO_Kick_Phi(2),PO_Kick_Scaling_Factor,PO_Kick_Theta(1),PO_Kick_Theta(2),PO_LBV_Factor,PO_LBV_Mass_Loss_Prscrptn,PO_MCBUR1,PO_MM_Kick_Multiplier_BH,PO_MM_Kick_Multiplier_NS,PO_MM_Sigma_Kick,PO_MT_Acc_Efficiency_Prscrptn,PO_MT_AngMom_Loss_Prscrptn,PO_MT_Crit_MR_Giant_Deg_Acc,PO_MT_Crit_MR_Giant_NonDeg_Acc,PO_MT_Crit_MR_HE_Giant_Deg_Acc,PO_MT_Crit_MR_HE_Giant_NonDeg_Acc,PO_MT_Crit_MR_HE_HG_Deg_Acc,PO_MT_Crit_MR_HE_HG_NonDeg_Acc,PO_MT_Crit_MR_HE_MS_Deg_Acc,PO_MT_Crit_MR_HE_MS_NonDeg_Acc,PO_MT_Crit_MR_HG_Deg_Acc,PO_MT_Crit_MR_HG_NonDeg_Acc,PO_MT_Crit_MR_MS_High_Mass_Deg_Acc,PO_MT_Crit_MR_MS_High_Mass_NonDeg_Acc,PO_MT_Crit_MR_MS_Low_Mass_Deg_Acc,PO_MT_Crit_MR_MS_Low_Mass_NonDeg_Acc,PO_MT_Crit_MR_WD_Deg_Acc,PO_MT_Crit_MR_WD_NonDeg_Acc,PO_MT_Fraction_Accreted,PO_MT_JLoss,PO_MT_JLoss_Macleod_Linear_Frac_Degen,PO_MT_JLoss_Macleod_Linear_Frac_Non_Degen,PO_MT_Rejuvenation_Prscrptn,PO_MT_Thermal_Limit_C,PO_MT_Thermally_Lmtd_Variation,PO_Mass_Loss_Prscrptn,PO_Mass_Ratio,PO_Mass_Ratio_Dstrbtn,PO_Mass_Ratio_Dstrbtn_Max,PO_Mass_Ratio_Dstrbtn_Min,PO_Max_Donor_Mass,PO_Max_Evolution_Time,PO_Max_NS_Mass,PO_Max_Timesteps,PO_Metallicity,PO_Metallicity_Dstrbtn,PO_Metallicity_Dstrbtn_Max,PO_Metallicity_Dstrbtn_Min,PO_Min_Secondary_Mass,PO_NS_EOS,PO_Neutrino_Mass_Loss_Assmptn,PO_Neutrino_Mass_Loss_Value,PO_Orbital_Period,PO_Orbital_Period_Dstrbtn,PO_Orbital_Period_Max,PO_Orbital_Period_Min,PO_Overall_WindMassLoss_Multipl,PO_PISN_Lower_Limit,PO_PISN_Upper_Limit,PO_PPI_CO_CORE_SHIFT_HENDRIKS,PO_PPI_Lower_Limit,PO_PPI_Prscrptn,PO_PPI_Upper_Limit,PO_Pulsar_Mag_Field_Decay_mScale,PO_Pulsar_Mag_Field_Decay_tScale,PO_Pulsar_Mag_Field_Dstrbtn,PO_Pulsar_Mag_Field_Dstrbtn_Max,PO_Pulsar_Mag_Field_Dstrbtn_Min,PO_Pulsar_Minimum_Mag_Field,PO_Pulsar_Spin_Period_Dstrbtn,PO_Pulsar_Spin_Period_Dstrbtn_Max,PO_Pulsar_Spin_Period_Dstrbtn_Min,PO_Remnant_Mass_Prscrptn,PO_Rocket_Kick_Magnitude(1),PO_Rocket_Kick_Magnitude(2),PO_Rocket_Kick_Phi(1),PO_Rocket_Kick_Phi(2),PO_Rocket_Kick_Theta(1),PO_Rocket_Kick_Theta(2),PO_Rotational_Frequency,PO_Rotational_Frequency(1),PO_Rotational_Frequency(2),PO_Rotational_Velocity_Dstrbtn,PO_SEED(CMDLINE),PO_SEED(OPTION),PO_Scale_CHE_Mass_Loss_Surf_He_Abundance,PO_Scale_Terminal_Wind_Vel_Metallicity_Power,PO_Semi-Major_Axis,PO_Semi-Major_Axis_Dstrbtn,PO_Semi-Major_Axis_Dstrbtn_Max,PO_Semi-Major_Axis_Dstrbtn_Min,PO_Sigma_Kick_CCSN_BH,PO_Sigma_Kick_CCSN_NS,PO

In [10]:
SP_columns = WD_SP_edit.columns

#let's see what column names overlap
set(dco_columns) & set(SP_columns)


{'Record_Type', 'SEED', 'Stellar_Type(1)', 'Stellar_Type(2)', 'mixture_weight'}

Let's select for systems that do not experience a stellar merger just to make sure that we are draling with systems that experience a merger due to gravitational waves

In [12]:
# stellar_merger_false = WD_SP_edit['Merger']==False
# WD_SP_edit_GW = WD_SP_edit[stellar_merger_false]

This is four less columns than the DCO table because I must add the coalescence time column as well as the merges in hubble time column (and we do not have recycled NS information)

#### Let's calculate the coalescence time and merges hubble time columns

Before doing this though we need to make sure we are only calculating these parameters for WDWD

In [17]:
# let's now create a dataframe where the above criteria is met based on all of the bools
WDWD_mask = np.logical_and(np.isin(WD_SP_edit['Stellar_Type(1)'],[10,11,12]),np.isin(WD_SP_edit['Stellar_Type(2)'],[10,11,12]))
WDWD_EDIT_SYS = WD_SP_edit[WDWD_mask]
WDWD_EDIT_SYS

,CH_on_MS(1),CH_on_MS(2),Eccentricity@ZAMS,Equilibrated_At_Birth,Error,Evolution_Status,Mass@ZAMS(1),Mass@ZAMS(2),Merger,Merger_At_Birth,Metallicity@ZAMS(1),Metallicity@ZAMS(2),Omega@ZAMS(1),Omega@ZAMS(2),PO_Add_Options_To_SysParms,PO_Allow_Immediate_RLOF>CE_To_Survive_CE,PO_Allow_MS_To_Survive_CE,PO_Allow_Non_Stripped_ECSN,PO_Allow_RLOF@Birth,PO_Allow_Radiative_Envelope_To_Survive_CE,PO_Allow_Touching@Birth,PO_BB_Mass_xFer_Stblty_Prscrptn,PO_BH_Kicks_Mode,PO_CE_Alpha,PO_CE_Alpha_Thermal,PO_CE_Formalism,PO_CE_Lambda,PO_CE_Lambda_Multiplier,PO_CE_Lambda_Prscrptn,PO_CE_Mass_Accr_Constant,PO_CE_Mass_Accr_Max,PO_CE_Mass_Accr_Min,PO_CE_Mass_Accr_Prscrptn,PO_CE_Recomb_Enrgy_Dnsty,PO_CE_Slope_Kruckow,PO_CHE_Mode,PO_Check_Photon_Tiring_Limit,PO_Circularise@MT,PO_Conserve_AngMom@Circ,PO_Cool_WindMassLoss_Multipl,PO_Eccentricity,PO_Eccentricity_Dstrbtn,PO_Eccentricity_Dstrbtn_Max,PO_Eccentricity_Dstrbtn_Min,PO_Eddington_Accr_Factor,PO_Enable_Rotationally_Enhanced_Mass_Loss,PO_Enhance_CHE_lifetimes_luminosities,PO_Envelope_State_Prscrptn,PO_Evolution_Mode,PO_FP_Error_Mode,PO_Fryer22_crit_COcore_Mass,PO_Fryer22_mixing_fraction,PO_Fryer_SN_Engine,PO_Initial_Mass,PO_Initial_Mass(1),PO_Initial_Mass(2),PO_Initial_Mass_Func_Max,PO_Initial_Mass_Func_Min,PO_Initial_Mass_Func_Power,PO_Initial_Mass_Function,PO_Kick_Direction_Dstrbtn,PO_Kick_Direction_Power,PO_Kick_Magnitude,PO_Kick_Magnitude(1),PO_Kick_Magnitude(2),PO_Kick_Magnitude_Dstrbtn,PO_Kick_Magnitude_Dstrbtn_Max,PO_Kick_Magnitude_Random,PO_Kick_Magnitude_Random(1),PO_Kick_Magnitude_Random(2),PO_Kick_Mean_Anomaly(1),PO_Kick_Mean_Anomaly(2),PO_Kick_Phi(1),PO_Kick_Phi(2),PO_Kick_Scaling_Factor,PO_Kick_Theta(1),PO_Kick_Theta(2),PO_LBV_Factor,PO_LBV_Mass_Loss_Prscrptn,PO_MCBUR1,PO_MM_Kick_Multiplier_BH,PO_MM_Kick_Multiplier_NS,PO_MM_Sigma_Kick,PO_MT_Acc_Efficiency_Prscrptn,PO_MT_AngMom_Loss_Prscrptn,PO_MT_Crit_MR_Giant_Deg_Acc,PO_MT_Crit_MR_Giant_NonDeg_Acc,PO_MT_Crit_MR_HE_Giant_Deg_Acc,PO_MT_Crit_MR_HE_Giant_NonDeg_Acc,PO_MT_Crit_MR_HE_HG_Deg_Acc,PO_MT_Crit_MR_HE_HG_NonDeg_Acc,PO_MT_Crit_MR_HE_MS_Deg_Acc,PO_MT_Crit_MR_HE_MS_NonDeg_Acc,PO_MT_Crit_MR_HG_Deg_Acc,PO_MT_Crit_MR_HG_NonDeg_Acc,PO_MT_Crit_MR_MS_High_Mass_Deg_Acc,PO_MT_Crit_MR_MS_High_Mass_NonDeg_Acc,PO_MT_Crit_MR_MS_Low_Mass_Deg_Acc,PO_MT_Crit_MR_MS_Low_Mass_NonDeg_Acc,PO_MT_Crit_MR_WD_Deg_Acc,PO_MT_Crit_MR_WD_NonDeg_Acc,PO_MT_Fraction_Accreted,PO_MT_JLoss,PO_MT_JLoss_Macleod_Linear_Frac_Degen,PO_MT_JLoss_Macleod_Linear_Frac_Non_Degen,PO_MT_Rejuvenation_Prscrptn,PO_MT_Thermal_Limit_C,PO_MT_Thermally_Lmtd_Variation,PO_Mass_Loss_Prscrptn,PO_Mass_Ratio,PO_Mass_Ratio_Dstrbtn,PO_Mass_Ratio_Dstrbtn_Max,PO_Mass_Ratio_Dstrbtn_Min,PO_Max_Donor_Mass,PO_Max_Evolution_Time,PO_Max_NS_Mass,PO_Max_Timesteps,PO_Metallicity,PO_Metallicity_Dstrbtn,PO_Metallicity_Dstrbtn_Max,PO_Metallicity_Dstrbtn_Min,PO_Min_Secondary_Mass,PO_NS_EOS,PO_Neutrino_Mass_Loss_Assmptn,PO_Neutrino_Mass_Loss_Value,PO_Orbital_Period,PO_Orbital_Period_Dstrbtn,PO_Orbital_Period_Max,PO_Orbital_Period_Min,PO_Overall_WindMassLoss_Multipl,PO_PISN_Lower_Limit,PO_PISN_Upper_Limit,PO_PPI_CO_CORE_SHIFT_HENDRIKS,PO_PPI_Lower_Limit,PO_PPI_Prscrptn,PO_PPI_Upper_Limit,PO_Pulsar_Mag_Field_Decay_mScale,PO_Pulsar_Mag_Field_Decay_tScale,PO_Pulsar_Mag_Field_Dstrbtn,PO_Pulsar_Mag_Field_Dstrbtn_Max,PO_Pulsar_Mag_Field_Dstrbtn_Min,PO_Pulsar_Minimum_Mag_Field,PO_Pulsar_Spin_Period_Dstrbtn,PO_Pulsar_Spin_Period_Dstrbtn_Max,PO_Pulsar_Spin_Period_Dstrbtn_Min,PO_Remnant_Mass_Prscrptn,PO_Rocket_Kick_Magnitude(1),PO_Rocket_Kick_Magnitude(2),PO_Rocket_Kick_Phi(1),PO_Rocket_Kick_Phi(2),PO_Rocket_Kick_Theta(1),PO_Rocket_Kick_Theta(2),PO_Rotational_Frequency,PO_Rotational_Frequency(1),PO_Rotational_Frequency(2),PO_Rotational_Velocity_Dstrbtn,PO_SEED(CMDLINE),PO_SEED(OPTION),PO_Scale_CHE_Mass_Loss_Surf_He_Abundance,PO_Scale_Terminal_Wind_Vel_Metallicity_Power,PO_Semi-Major_Axis,PO_Semi-Major_Axis_Dstrbtn,PO_Semi-Major_Axis_Dstrbtn_Max,PO_Semi-Major_Axis_Dstrbtn_Min,PO_Sigma_Kick_CCSN_BH,PO_Sigma_Kick_CCSN_NS,PO

In [18]:
print(np.unique(WDWD_EDIT_SYS['Stellar_Type(1)']))
print(np.unique(WDWD_EDIT_SYS['Stellar_Type(2)']))

[11 12]
[10 11 12]


In the equation for coalescence time, M1 refers to the more massive star whereas M2 refers to the less massive one. I need to make new columns (witht eh help of Lieke) to find these more massive and less massive stars in each bianry. Let's do a check to see if our colaescence times match the COMPAS calculations

In [34]:
# unit_dict = {b'Rsol':u.Rsun,
#              b'AU':u.AU}

# print(unit_dict[b'AU'])

In [19]:
# # let's first look at the units of some parameters to see if anything must be converted
# SPs_WD = Data_WD_edit['BSE_System_Parameters']
# print(SPs_WD['Mass(1)'].attrs['units']) 
# print(SPs_WD['SemiMajorAxis'].attrs['units'])
# print(SPs_WD['Time'].attrs['units'])

In [20]:
# # let's first look at the units of some parameters to see if anything must be converted
# dcos_WD = Data_WD_edit['BSE_Double_Compact_Objects']
# print(dcos_WD['Mass(1)'].attrs['units']) 
# print(dcos_WD['SemiMajorAxis@DCO'].attrs['units'])
# print(dcos_WD['Time'].attrs['units'])

In [69]:
# # looking at how accurate the coaslescence calulations are
# mergingDCO = DCO_OG[DCO_OG['Merges_Hubble_Time'] == 1]

# # Add columns for the more and less massive compact object
# mergingDCO['M_moremass'] = mergingDCO[['Mass(1)', 'Mass(2)']].max(axis=1)
# mergingDCO['M_lessmass'] = mergingDCO[['Mass(1)', 'Mass(2)']].min(axis=1)

# time_gw = tgw((mergingDCO['SemiMajorAxis@DCO'].values),(mergingDCO['Eccentricity@DCO']),(mergingDCO['M_moremass'].values),(mergingDCO['M_lessmass'].values),Data_WD_edit,'BSE_Double_Compact_Objects','SemiMajorAxis@DCO')

# plt.scatter(mergingDCO['SEED'], abs(time_gw - mergingDCO['Coalescence_Time']) )
# plt.xlabel("SEED")
# plt.ylabel('absolute difference calc t_col and COMPAS Coalescence_Time ')
# plt.yscale('log')
# plt.title("Difference in Computation having separations = [Rsun]")
# plt.show()

#### Let's now calculated the coalescence time to our table of WD+WD

In [37]:
# Add columns for the more and less massive compact object
# WDWD_EDIT_SYS['M_moremass'] = WDWD_EDIT_SYS[['Mass(1)', 'Mass(2)']].max(axis=1)
# WDWD_EDIT_SYS['M_lessmass'] = WDWD_EDIT_SYS[['Mass(1)', 'Mass(2)']].min(axis=1)

M1 = np.maximum(WDWD_EDIT_SYS['Mass@ZAMS(1)'],WDWD_EDIT_SYS['Mass@ZAMS(2)'])
M2 = np.minimum(Mass1, Mass2)

/tmp/ipykernel_4976/3822573989.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WDWD_EDIT_SYS['M_moremass'] = WDWD_EDIT_SYS[['Mass(1)', 'Mass(2)']].max(axis=1)
/tmp/ipykernel_4976/3822573989.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WDWD_EDIT_SYS['M_lessmass'] = WDWD_EDIT_SYS[['Mass(1)', 'Mass(2)']].min(axis=1)


In [38]:
# let's calculate the coalescence time in Myr
time_col_wd = useful_fncs.tgw((WDWD_EDIT_SYS['SemiMajorAxis'].values),(WDWD_EDIT_SYS['Eccentricity']),(WDWD_EDIT_SYS['M_moremass'].values),(WDWD_EDIT_SYS['M_lessmass'].values),Data_WD_edit,'BSE_System_Parameters','SemiMajorAxis')

/srv/conda/envs/notebook/lib/python3.10/site-packages/astropy/units/quantity.py:671: RuntimeWarning: divide by zero encountered in divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/srv/conda/envs/notebook/lib/python3.10/site-packages/astropy/units/quantity.py:671: RuntimeWarning: invalid value encountered in divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


In [39]:
# let's insert the check if there was a merger before hubble time into our dataframe
# only run this cell once per table creation
WDWD_EDIT_SYS.insert(1,"Coalescence_Time",time_col_wd,True)
WDWD_EDIT_SYS

,CE_Event_Counter,Coalescence_Time,Eccentricity,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merger,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,SEED,SemiMajorAxis,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight,M_moremass,M_lessmass
0,0,6.717356e+21,0.000000,0,b'NA ',b'NA ',0.679584,0.745539,0,0.005205,0,1,0,75369.460455,11,1,13940.060611,9.915128,0.745539,0.679584
1,0,3.251682e+23,0.000000,0,b'NA ',b'NA ',0.890890,0.558981,0,0.000876,0,1,1,198801.165570,11,11,9025.539968,0.245931,0.890890,0.558981
2,0,NaN,5.388305,0,b'NA ',b'NA ',1.213017,0.733545,0,0.000696,0,1,2,-74.873498,13,11,13767.520389,0.392874,1.213017,0.733545
3,0,3.949743e+17,0.000000,0,b'NA ',b'NA ',0.553602,0.180391,0,0.006523,0,1,3,3725.397689,11,0,14176.819295,10.000000,0.553602,0.180391
7,1,7.663460e+04,0.000000,0,b'5 ',b'NA ',0.595781,0.199647,0,0.001113,0,1,7,2.635409,11,0,13776.746205,10.000000,0.595781,0.199647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0,6.959760e+21,0.000000,0,b'NA ',b'NA ',1.275749,0.658100,0,0.029347,0,1,999995,93115.659434,11,11,1156.571375,0.094042,1.275749,0.658100
999996,0,6.594915e+18,0.000000,0,b'NA ',b'NA ',0.780910,0.722761,0,0.014557,0,1,999996,13891.321460,11,11,913.213610,0.151815,0.780910,0.722761
999997,0,1.988156e+19,0.000000,0,b'NA ',b'NA ',1.381447,0.667080,0,0.016232,0,1,999997,22354.008782,12,11,1503.425822,0.234124,1.381447,0.667080
999998,0,5.556545e+19,0.000000,0,b'NA ',b'NA ',0.942527,0.770343,0,0.003586,0,1,999998,26039.406126,11,11,742.013427,0.115753,0.942527,0.770343


In [40]:
# let's drop all the systems that produce NA for their coalescence time for now (we will come back to why these systems are like this)
WDWD_EDIT_SYS = WDWD_EDIT_SYS.dropna(subset=['Coalescence_Time'])

In [74]:
# in case you run the cell above more than once here is a way to remove the extra column
# WDWD_EDIT_SYS = WDWD_EDIT_SYS.drop('Coalescence_Time', axis=1)

We now need to calcualte the delay time to then see if this time is within a hubble time

In [41]:
# let's convert the columns of the data frame into numpy arrays to be able to do operations with them
time = np.array(WDWD_EDIT_SYS['Time']) # in Myr
t_col = np.array(WDWD_EDIT_SYS['Coalescence_Time']) # in Myr

t_delay = [] # this will hold the delay time in Myr

for binaries in range(WDWD_EDIT_SYS.shape[0]):

    delay = time[binaries] + t_col[binaries] # adding the two values together to get the delay time
    t_delay.append(delay)

In [42]:
# first let's take the delay time column and make it a numpy array
delay_time = np.array(t_delay)

# the age of the universe
age_universe = (13.7e9)*(1e-6) # converting from yr to Myr

hubble_merger = []

for times in delay_time:

    if (times > age_universe):
        hubble_merger.append(0)

    elif (times < age_universe):
        hubble_merger.append(1)

In [43]:
# let's insert the check if there was a merger before hubble time into our dataframe

# only run this cell once per table creation
WDWD_EDIT_SYS.insert(8,"Merges_Hubble_Time",hubble_merger,True)

In [78]:
# in case you need to remove the column
# WDWD_SYS = WDWD_SYS.drop('Merges_Hubble_Time', axis=1)

In [44]:
WDWD_EDIT_SYS

,CE_Event_Counter,Coalescence_Time,Eccentricity,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merges_Hubble_Time,Merger,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,SEED,SemiMajorAxis,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight,M_moremass,M_lessmass
0,0,6.717356e+21,0.0,0,b'NA ',b'NA ',0.679584,0.745539,0,0,0.005205,0,1,0,75369.460455,11,1,13940.060611,9.915128,0.745539,0.679584
1,0,3.251682e+23,0.0,0,b'NA ',b'NA ',0.890890,0.558981,0,0,0.000876,0,1,1,198801.165570,11,11,9025.539968,0.245931,0.890890,0.558981
3,0,3.949743e+17,0.0,0,b'NA ',b'NA ',0.553602,0.180391,0,0,0.006523,0,1,3,3725.397689,11,0,14176.819295,10.000000,0.553602,0.180391
7,1,7.663460e+04,0.0,0,b'5 ',b'NA ',0.595781,0.199647,0,0,0.001113,0,1,7,2.635409,11,0,13776.746205,10.000000,0.595781,0.199647
9,0,6.963150e+20,0.0,0,b'NA ',b'NA ',1.102787,1.102454,0,0,0.004591,0,1,9,59365.315016,11,11,157.279070,0.152716,1.102787,1.102454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0,6.959760e+21,0.0,0,b'NA ',b'NA ',1.275749,0.658100,0,0,0.029347,0,1,999995,93115.659434,11,11,1156.571375,0.094042,1.275749,0.658100
999996,0,6.594915e+18,0.0,0,b'NA ',b'NA ',0.780910,0.722761,0,0,0.014557,0,1,999996,13891.321460,11,11,913.213610,0.151815,0.780910,0.722761
999997,0,1.988156e+19,0.0,0,b'NA ',b'NA ',1.381447,0.667080,0,0,0.016232,0,1,999997,22354.008782,12,11,1503.425822,0.234124,1.381447,0.667080
999998,0,5.556545e+19,0.0,0,b'NA ',b'NA ',0.942527,0.770343,0,0,0.003586,0,1,999998,26039.406126,11,11,742.013427,0.115753,0.942527,0.770343


Let's look at where the \"Merges_Hubble_Time\" is set to true

In [45]:
# there is only one system that merges before hubble time
WDWD_Merge = WDWD_EDIT_SYS[WDWD_EDIT_SYS['Merges_Hubble_Time']==True]
WDWD_Merge

,CE_Event_Counter,Coalescence_Time,Eccentricity,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merges_Hubble_Time,Merger,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,SEED,SemiMajorAxis,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight,M_moremass,M_lessmass
29,1,64.106710,0.0,0,b'8 ',b'3 ',0.483516,0.394784,1,0,0.000438,0,1,29,0.517110,11,11,1536.080151,7.360919,0.483516,0.394784
79,2,0.008269,0.0,1,b'5 ',b'3 ',0.629323,0.227306,1,1,0.000261,0,1,79,0.050954,11,10,3502.246069,8.902695,0.629323,0.227306
86,2,0.049491,0.0,0,b'5 ',b'3 ',0.568929,0.211294,1,0,0.002140,0,1,86,0.074546,11,10,8874.314167,7.842265,0.568929,0.211294
87,1,8321.641517,0.0,0,b'3 ',b'5 ',0.320580,0.526926,1,0,0.002418,0,1,87,1.677890,10,11,4428.913115,10.000000,0.526926,0.320580
97,1,441.783850,0.0,0,b'1-2 ',b'5-8 ',0.276373,0.679873,1,0,0.012405,0,1,97,0.852472,10,11,1390.603090,9.999991,0.679873,0.276373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999761,2,2834.654788,0.0,0,b'5 ',b'3 ',0.770769,0.396774,1,0,0.013625,0,1,999761,1.610884,11,11,940.615944,0.254697,0.770769,0.396774
999848,1,0.006538,0.0,1,b'2-8 ',b'3 ',0.783044,1.002699,1,1,0.001370,0,1,999848,0.088370,11,7,118.537793,0.305238,1.002699,0.783044
999871,2,1073.100464,0.0,0,b'6 ',b'5 ',0.954424,0.569040,1,0,0.000369,0,1,999871,1.558990,11,11,1787.995323,0.212650,0.954424,0.569040
999912,1,3009.383390,0.0,0,b'2-8 ',b'5-8 ',0.773363,0.658614,1,0,0.004168,0,1,999912,1.954841,11,11,382.472452,0.160558,0.773363,0.658614


In [81]:
# # making sure all of these systems do not experience a stellar merger
# bool_merge = WDWD_Merge['Merger']==False
# sum(bool_merge)

In [46]:
# let's drop the merger column
WDWD_Merge = WDWD_Merge.drop('Merger', axis=1)

#### Let's add our systems that merged within a hubble time to the hdf5 file

Before concatenating the systems to the original DCO table we need to rename some of the columns in our WD table so that they match up with the DCO table

In [47]:
WDWD_Merge = WDWD_Merge.rename(columns={'Eccentricity':'Eccentricity@DCO','SemiMajorAxis':'SemiMajorAxis@DCO'})

In [48]:
WDWD_Merge

,CE_Event_Counter,Coalescence_Time,Eccentricity@DCO,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merges_Hubble_Time,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,SEED,SemiMajorAxis@DCO,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight,M_moremass,M_lessmass
29,1,64.106710,0.0,0,b'8 ',b'3 ',0.483516,0.394784,1,0.000438,0,1,29,0.517110,11,11,1536.080151,7.360919,0.483516,0.394784
79,2,0.008269,0.0,1,b'5 ',b'3 ',0.629323,0.227306,1,0.000261,0,1,79,0.050954,11,10,3502.246069,8.902695,0.629323,0.227306
86,2,0.049491,0.0,0,b'5 ',b'3 ',0.568929,0.211294,1,0.002140,0,1,86,0.074546,11,10,8874.314167,7.842265,0.568929,0.211294
87,1,8321.641517,0.0,0,b'3 ',b'5 ',0.320580,0.526926,1,0.002418,0,1,87,1.677890,10,11,4428.913115,10.000000,0.526926,0.320580
97,1,441.783850,0.0,0,b'1-2 ',b'5-8 ',0.276373,0.679873,1,0.012405,0,1,97,0.852472,10,11,1390.603090,9.999991,0.679873,0.276373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999761,2,2834.654788,0.0,0,b'5 ',b'3 ',0.770769,0.396774,1,0.013625,0,1,999761,1.610884,11,11,940.615944,0.254697,0.770769,0.396774
999848,1,0.006538,0.0,1,b'2-8 ',b'3 ',0.783044,1.002699,1,0.001370,0,1,999848,0.088370,11,7,118.537793,0.305238,1.002699,0.783044
999871,2,1073.100464,0.0,0,b'6 ',b'5 ',0.954424,0.569040,1,0.000369,0,1,999871,1.558990,11,11,1787.995323,0.212650,0.954424,0.569040
999912,1,3009.383390,0.0,0,b'2-8 ',b'5-8 ',0.773363,0.658614,1,0.004168,0,1,999912,1.954841,11,11,382.472452,0.160558,0.773363,0.658614


In [49]:
DCO_appended_table = pd.concat([DCO_OG,WDWD_Merge])

In [50]:
DCO_appended_table

,CE_Event_Counter,Coalescence_Time,Eccentricity@DCO,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merges_Hubble_Time,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,Recycled_NS(1),Recycled_NS(2),SEED,SemiMajorAxis@DCO,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight,M_moremass,M_lessmass
0,0,2.328412e+13,1.110223e-16,0,b'4 ',b'NA ',31.911977,27.289501,0,0.000473,0,1,0.0,0.0,158,43.919694,14,14,4.347694,10.000000,NaN,NaN
1,0,3.974767e+19,5.431362e-02,0,b'NA ',b'NA ',5.843895,5.255385,0,0.000179,0,1,0.0,0.0,326,453.863126,14,14,16.329921,10.000000,NaN,NaN
2,1,6.281330e+02,5.821410e-01,0,b'2 ',b'4-8 ',9.648989,1.454633,1,0.001588,0,1,0.0,0.0,695,0.033701,14,13,14.260153,10.000000,NaN,NaN
3,1,5.402979e+03,9.111420e-01,0,b'4 ',b'5-8 ',3.382412,1.470501,1,0.000260,0,1,0.0,0.0,858,0.114685,14,13,19.579543,10.000000,NaN,NaN
4,0,8.206604e+18,6.933256e-01,0,b'NA ',b'NA ',10.933214,2.614135,0,0.001677,0,1,0.0,0.0,948,557.934571,14,14,31.652154,10.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999761,2,2.834655e+03,0.000000e+00,0,b'5 ',b'3 ',0.770769,0.396774,1,0.013625,0,1,NaN,NaN,999761,1.610884,11,11,940.615944,0.254697,0.770769,0.396774
999848,1,6.537726e-03,0.000000e+00,1,b'2-8 ',b'3 ',0.783044,1.002699,1,0.001370,0,1,NaN,NaN,999848,0.088370,11,7,118.537793,0.305238,1.002699,0.783044
999871,2,1.073100e+03,0.000000e+00,0,b'6 ',b'5 ',0.954424,0.569040,1,0.000369,0,1,NaN,NaN,999871,1.558990,11,11,1787.995323,0.212650,0.954424,0.569040
999912,1,3.009383e+03,0.000000e+00,0,b'2-8 ',b'5-8 ',0.773363,0.658614,1,0.004168,0,1,NaN,NaN,999912,1.954841,11,11,382.472452,0.160558,0.773363,0.658614


In [51]:
# let's drop the extra 
DCO_appended_table = DCO_appended_table.drop('M_moremass', axis=1)
DCO_appended_table = DCO_appended_table.drop('M_lessmass', axis=1)

In [52]:
DCO_appended_table

,CE_Event_Counter,Coalescence_Time,Eccentricity@DCO,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merges_Hubble_Time,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,Recycled_NS(1),Recycled_NS(2),SEED,SemiMajorAxis@DCO,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight
0,0,2.328412e+13,1.110223e-16,0,b'4 ',b'NA ',31.911977,27.289501,0,0.000473,0,1,0.0,0.0,158,43.919694,14,14,4.347694,10.000000
1,0,3.974767e+19,5.431362e-02,0,b'NA ',b'NA ',5.843895,5.255385,0,0.000179,0,1,0.0,0.0,326,453.863126,14,14,16.329921,10.000000
2,1,6.281330e+02,5.821410e-01,0,b'2 ',b'4-8 ',9.648989,1.454633,1,0.001588,0,1,0.0,0.0,695,0.033701,14,13,14.260153,10.000000
3,1,5.402979e+03,9.111420e-01,0,b'4 ',b'5-8 ',3.382412,1.470501,1,0.000260,0,1,0.0,0.0,858,0.114685,14,13,19.579543,10.000000
4,0,8.206604e+18,6.933256e-01,0,b'NA ',b'NA ',10.933214,2.614135,0,0.001677,0,1,0.0,0.0,948,557.934571,14,14,31.652154,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999761,2,2.834655e+03,0.000000e+00,0,b'5 ',b'3 ',0.770769,0.396774,1,0.013625,0,1,NaN,NaN,999761,1.610884,11,11,940.615944,0.254697
999848,1,6.537726e-03,0.000000e+00,1,b'2-8 ',b'3 ',0.783044,1.002699,1,0.001370,0,1,NaN,NaN,999848,0.088370,11,7,118.537793,0.305238
999871,2,1.073100e+03,0.000000e+00,0,b'6 ',b'5 ',0.954424,0.569040,1,0.000369,0,1,NaN,NaN,999871,1.558990,11,11,1787.995323,0.212650
999912,1,3.009383e+03,0.000000e+00,0,b'2-8 ',b'5-8 ',0.773363,0.658614,1,0.004168,0,1,NaN,NaN,999912,1.954841,11,11,382.472452,0.160558


In [53]:
DCO_appended_table.sort_values(by="SEED")

,CE_Event_Counter,Coalescence_Time,Eccentricity@DCO,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merges_Hubble_Time,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,Recycled_NS(1),Recycled_NS(2),SEED,SemiMajorAxis@DCO,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight
29,1,64.106710,0.0,0,b'8 ',b'3 ',0.483516,0.394784,1,0.000438,0,1,NaN,NaN,29,0.517110,11,11,1536.080151,7.360919
79,2,0.008269,0.0,1,b'5 ',b'3 ',0.629323,0.227306,1,0.000261,0,1,NaN,NaN,79,0.050954,11,10,3502.246069,8.902695
86,2,0.049491,0.0,0,b'5 ',b'3 ',0.568929,0.211294,1,0.002140,0,1,NaN,NaN,86,0.074546,11,10,8874.314167,7.842265
87,1,8321.641517,0.0,0,b'3 ',b'5 ',0.320580,0.526926,1,0.002418,0,1,NaN,NaN,87,1.677890,10,11,4428.913115,10.000000
97,1,441.783850,0.0,0,b'1-2 ',b'5-8 ',0.276373,0.679873,1,0.012405,0,1,NaN,NaN,97,0.852472,10,11,1390.603090,9.999991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999761,2,2834.654788,0.0,0,b'5 ',b'3 ',0.770769,0.396774,1,0.013625,0,1,NaN,NaN,999761,1.610884,11,11,940.615944,0.254697
999848,1,0.006538,0.0,1,b'2-8 ',b'3 ',0.783044,1.002699,1,0.001370,0,1,NaN,NaN,999848,0.088370,11,7,118.537793,0.305238
999871,2,1073.100464,0.0,0,b'6 ',b'5 ',0.954424,0.569040,1,0.000369,0,1,NaN,NaN,999871,1.558990,11,11,1787.995323,0.212650
999912,1,3009.383390,0.0,0,b'2-8 ',b'5-8 ',0.773363,0.658614,1,0.004168,0,1,NaN,NaN,999912,1.954841,11,11,382.472452,0.160558


In [54]:
# let's save this table as a csv so we can add it to the edited hdf5 file

DCO_appended_table.to_csv('Edit_DCO_ALL.csv',index=False)

In [55]:
Data_WD_edit.close()

### Let's now rewrite the hdf5 file so that we can run FastCosmicIntegration on it

In [56]:
# let's create a group that we wil called BSE_DCOs using out csv file

# Example CSV file path
csv_file = 'Edit_DCO_ALL.csv'

# Open or create an HDF5 file
with h5.File(pathToWDWD_edit_H5, 'a') as hdf_file:
    
    # deleting the previous DCO group
    del hdf_file['BSE_Double_Compact_Objects']
    print('Original BSE_Double_Compact_Objects Group deleted.')

    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)
    
    # Create a group (if not already created)
    group_name = 'BSE_Double_Compact_Objects'
    if group_name not in hdf_file:
        group = hdf_file.create_group(group_name)
    else:
        group = hdf_file[group_name]

   # Create datasets for each column in the CSV file
    for column in df.columns:
        dataset_name = column


        # Handle existing dataset
        if dataset_name in group:

            # Option 1: Overwrite existing dataset
            del group[dataset_name]
            dataset = group.create_dataset(dataset_name,data=df[column].to_numpy())

            # Option 2: Rename existing dataset
            # new_dataset_name = f"{dataset_name}_new"
            # dataset = group.create_dataset(new_dataset_name, data=df[column].to_numpy())

        else:
            # Create dataset and write data from DataFrame
            dataset = group.create_dataset(dataset_name, data=df[column].to_numpy())
            
            # Optionally, add attributes to the dataset (if needed)
            dataset.attrs['description'] = f'Data for column {column} from CSV file'
    
print(f'CSV data from {csv_file} added to group "{group_name}" in HDF5 file: {pathToWDWD_edit_H5}')

Original BSE_Double_Compact_Objects Group deleted.
CSV data from Edit_DCO_ALL.csv added to group "BSE_Double_Compact_Objects" in HDF5 file: /home/jovyan/home/edit_hdf5/WDWD/v02.46.01/COMPAS_Output_wWeights_COWD_DD.h5


In [57]:
# reading in the HDF5 file - this file is the AIS file that we should expect many WDs in
pathToWDWD_edit_H5 = '/home/jovyan/home/edit_hdf5/WDWD/v02.46.01/COMPAS_Output_wWeights_COWD_DD.h5' #path of the hdf5 file
Data_WD_edit = h5.File(pathToWDWD_edit_H5)

Let's check if this new group has been added to the hdf5 file

In [58]:
Data_WD_edit.keys()

<KeysViewHDF5 ['BSE_Common_Envelopes', 'BSE_Double_Compact_Objects', 'BSE_Pulsar_Evolution', 'BSE_RLOF', 'BSE_Supernovae', 'BSE_System_Parameters', 'Run_Details']>

Let's now check to see if the correct dataset names are present

In [59]:
DCOWDs = Data_WD_edit['BSE_Double_Compact_Objects'] #specifically looking at the supernovae events
list(DCOWDs.keys()) #listing the parameters recorded for each supernova

['CE_Event_Counter',
 'Coalescence_Time',
 'Eccentricity@DCO',
 'Immediate_RLOF>CE',
 'MT_Donor_Hist(1)',
 'MT_Donor_Hist(2)',
 'Mass(1)',
 'Mass(2)',
 'Merges_Hubble_Time',
 'Metallicity@ZAMS(1)',
 'Optimistic_CE',
 'Record_Type',
 'Recycled_NS(1)',
 'Recycled_NS(2)',
 'SEED',
 'SemiMajorAxis@DCO',
 'Stellar_Type(1)',
 'Stellar_Type(2)',
 'Time',
 'mixture_weight']

In [60]:
# let's open this group 
# we want to read in the bse_system_paramtetrs to get information about thes different systems
DCO_WD = pd.DataFrame() # making a pandas dataframe
for key in Data_WD_edit["BSE_Double_Compact_Objects"].keys(): #looping through the \"keys\" or paramters in BSE_System_Parameters
    DCO_WD[key] = Data_WD_edit["BSE_Double_Compact_Objects"][key][()] # adding these columns to the dataframe"

In [61]:
DCO_WD

,CE_Event_Counter,Coalescence_Time,Eccentricity@DCO,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merges_Hubble_Time,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,Recycled_NS(1),Recycled_NS(2),SEED,SemiMajorAxis@DCO,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight
0,0,2.328412e+13,1.110223e-16,0,"b""b'4 '""","b""b'NA '""",31.911977,27.289501,0,0.000473,0,1,0.0,0.0,158,43.919694,14,14,4.347694,10.000000
1,0,3.974767e+19,5.431362e-02,0,"b""b'NA '""","b""b'NA '""",5.843895,5.255385,0,0.000179,0,1,0.0,0.0,326,453.863126,14,14,16.329921,10.000000
2,1,6.281330e+02,5.821410e-01,0,"b""b'2 '""","b""b'4-8 '""",9.648989,1.454633,1,0.001588,0,1,0.0,0.0,695,0.033701,14,13,14.260153,10.000000
3,1,5.402979e+03,9.111420e-01,0,"b""b'4 '""","b""b'5-8 '""",3.382412,1.470501,1,0.000260,0,1,0.0,0.0,858,0.114685,14,13,19.579543,10.000000
4,0,8.206604e+18,6.933256e-01,0,"b""b'NA '""","b""b'NA '""",10.933214,2.614135,0,0.001677,0,1,0.0,0.0,948,557.934571,14,14,31.652154,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21684,2,2.834655e+03,0.000000e+00,0,"b""b'5 '""","b""b'3 '""",0.770769,0.396774,1,0.013625,0,1,NaN,NaN,999761,1.610884,11,11,940.615944,0.254697
21685,1,6.537726e-03,0.000000e+00,1,"b""b'2-8 '""","b""b'3 '""",0.783044,1.002699,1,0.001370,0,1,NaN,NaN,999848,0.088370,11,7,118.537793,0.305238
21686,2,1.073100e+03,0.000000e+00,0,"b""b'6 '""","b""b'5 '""",0.954424,0.569040,1,0.000369,0,1,NaN,NaN,999871,1.558990,11,11,1787.995323,0.212650
21687,1,3.009383e+03,0.000000e+00,0,"b""b'2-8 '""","b""b'5-8 '""",0.773363,0.658614,1,0.004168,0,1,NaN,NaN,999912,1.954841,11,11,382.472452,0.160558


In [62]:
DCO_WD[DCO_WD['Stellar_Type(1)']==10]

,CE_Event_Counter,Coalescence_Time,Eccentricity@DCO,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merges_Hubble_Time,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,Recycled_NS(1),Recycled_NS(2),SEED,SemiMajorAxis@DCO,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight
466,1,8321.641517,0.0,0,"b""b'3 '""","b""b'5 '""",0.320580,0.526926,1,0.002418,0,1,NaN,NaN,87,1.677890,10,11,4428.913115,10.000000
467,1,441.783850,0.0,0,"b""b'1-2 '""","b""b'5-8 '""",0.276373,0.679873,1,0.012405,0,1,NaN,NaN,97,0.852472,10,11,1390.603090,9.999991
473,1,86.608924,0.0,0,"b""b'2 '""","b""b'5-8 '""",0.472254,0.654884,1,0.000938,0,1,NaN,NaN,260,0.669459,10,11,1568.332550,9.999465
481,1,17.143360,0.0,0,"b""b'1-2 '""","b""b'3-8 '""",0.582157,0.430288,1,0.017261,0,1,NaN,NaN,717,0.412402,10,11,393.755904,9.999963
490,1,100.860478,0.0,0,"b""b'3 '""","b""b'5 '""",0.310881,0.556748,1,0.000674,0,1,NaN,NaN,1008,0.563419,10,11,2815.047913,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19298,1,863.861584,0.0,0,"b""b'3 '""","b""b'6 '""",0.337729,0.570823,1,0.000852,0,1,NaN,NaN,880612,1.001660,10,11,2377.634811,8.368650
19665,1,55.585204,0.0,0,"b""b'1-2 '""","b""b'3-8 '""",0.358666,0.595628,1,0.000581,0,1,NaN,NaN,899939,0.524000,10,11,711.387798,2.279615
20590,1,110.091570,0.0,0,"b""b'1-2 '""","b""b'3-8 '""",0.316981,0.620999,1,0.000743,0,1,NaN,NaN,944275,0.606418,10,11,742.318838,2.978131
20748,1,4515.164570,0.0,0,"b""b'3 '""","b""b'6 '""",0.355875,0.587090,1,0.000191,0,1,NaN,NaN,952694,1.559722,10,11,3330.670820,4.263054


The CSV would not add to the hdf5 file if I did not make the dupicate columns but let's drop them now so we can run FastCosmicIntegration on this!

In [45]:
# DCO_WD = DCO_WD.drop(['CE_Event_Counter_new','Coalescence_Time_new','Eccentricity@DCO_new','Immediate_RLOF>CE_new','MT_Donor_Hist(1)_new', \
# 'MT_Donor_Hist(2)_new','Mass(1)_new','Mass(2)_new','Merges_Hubble_Time_new','Metallicity@ZAMS(1)_new','Optimistic_CE_new','Record_Type_new',\
# 'Recycled_NS(1)_new','Recycled_NS(2)_new','SEED_new','SemiMajorAxis@DCO_new','Stellar_Type(1)_new','Stellar_Type(2)_new','Time_new','mixture_weight_new'],axis=1)

In [63]:
DCO_WD

,CE_Event_Counter,Coalescence_Time,Eccentricity@DCO,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merges_Hubble_Time,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,Recycled_NS(1),Recycled_NS(2),SEED,SemiMajorAxis@DCO,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight
0,0,2.328412e+13,1.110223e-16,0,"b""b'4 '""","b""b'NA '""",31.911977,27.289501,0,0.000473,0,1,0.0,0.0,158,43.919694,14,14,4.347694,10.000000
1,0,3.974767e+19,5.431362e-02,0,"b""b'NA '""","b""b'NA '""",5.843895,5.255385,0,0.000179,0,1,0.0,0.0,326,453.863126,14,14,16.329921,10.000000
2,1,6.281330e+02,5.821410e-01,0,"b""b'2 '""","b""b'4-8 '""",9.648989,1.454633,1,0.001588,0,1,0.0,0.0,695,0.033701,14,13,14.260153,10.000000
3,1,5.402979e+03,9.111420e-01,0,"b""b'4 '""","b""b'5-8 '""",3.382412,1.470501,1,0.000260,0,1,0.0,0.0,858,0.114685,14,13,19.579543,10.000000
4,0,8.206604e+18,6.933256e-01,0,"b""b'NA '""","b""b'NA '""",10.933214,2.614135,0,0.001677,0,1,0.0,0.0,948,557.934571,14,14,31.652154,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21684,2,2.834655e+03,0.000000e+00,0,"b""b'5 '""","b""b'3 '""",0.770769,0.396774,1,0.013625,0,1,NaN,NaN,999761,1.610884,11,11,940.615944,0.254697
21685,1,6.537726e-03,0.000000e+00,1,"b""b'2-8 '""","b""b'3 '""",0.783044,1.002699,1,0.001370,0,1,NaN,NaN,999848,0.088370,11,7,118.537793,0.305238
21686,2,1.073100e+03,0.000000e+00,0,"b""b'6 '""","b""b'5 '""",0.954424,0.569040,1,0.000369,0,1,NaN,NaN,999871,1.558990,11,11,1787.995323,0.212650
21687,1,3.009383e+03,0.000000e+00,0,"b""b'2-8 '""","b""b'5-8 '""",0.773363,0.658614,1,0.004168,0,1,NaN,NaN,999912,1.954841,11,11,382.472452,0.160558


We can now use this group instead of BSE_Double_Compact_Objects to do the rate calculation!

In [49]:
# we want to read in the bse_system_paramtetrs to get information about thes different systems
WD_SP = pd.DataFrame() # making a pandas dataframe

for key in Data_WD_edit["BSE_System_Parameters"].keys(): #looping through the "keys" or paramters in BSE_System_Parameters
    WD_SP[key] = Data_WD_edit["BSE_System_Parameters"][key][()] # adding these columns to the dataframe

In [50]:
# making sure that the m1min = 0.9
print(min(WD_SP['Mass@ZAMS(1)']))
print(max(WD_SP['Mass@ZAMS(1)']))

0.9000149145946927
149.99316622286932


In [51]:
# # we want to read in the bse_system_paramtetrs to get information about thes different systems
# DETAILS = pd.DataFrame() # making a pandas dataframe

# for key in Data_WD_edit["Run_Details"].keys(): #looping through the "keys" or paramters in BSE_System_Parameters
#     DETAILS[key] = Data_WD_edit["Run_Details"][key][()] # adding these columns to the dataframe

In [52]:
# # pd.set_option('display.max_columns', None)
# # display(DETAILS.iloc[0])
# DETAILS['metallicity-distribution']

In [99]:
Data_WD_edit.close()